### Custom Layer example

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

### Load some data

In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### Define custom layer

The build() method should define two trainable weights α and β, both of shape input_shape[-1:] and data type tf.float32. α should be initialized with 1s, and β with 0s._

In [3]:
class LayerNormalization(keras.layers.Layer):
    def __init__(self, eps=0.001, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps

    def build(self, batch_input_shape):
        self.alpha = self.add_weight(
            name="alpha", shape=batch_input_shape[-1:],
            initializer="ones")
        self.beta = self.add_weight(
            name="beta", shape=batch_input_shape[-1:],
            initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    def call(self, X):
        mean, variance = tf.nn.moments(X, axes=-1, keepdims=True)
        return self.alpha * (X - mean) / (tf.sqrt(variance + self.eps)) + self.beta

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "eps": self.eps}

Exercise: Ensure that your custom layer produces the same (or very nearly the same) output as the keras.layers.LayerNormalization layer.

Let's create one instance of each class, apply them to some data, and ensure that the difference is negligeable.

In [4]:
X = housing.data.astype(np.float32)

my_layer = LayerNormalization()
keras_layer = keras.layers.LayerNormalization()

print(X.shape)
print(my_layer(X).shape)
print(keras_layer(X).shape)

difference = keras.losses.mean_absolute_error(keras_layer(X), my_layer(X))

print(difference)

tf.reduce_mean(difference)

(20640, 8)
(20640, 8)
(20640, 8)
tf.Tensor(
[1.9441359e-08 1.1175871e-08 5.0757080e-08 ... 4.6566129e-08 5.0291419e-08
 5.5879354e-08], shape=(20640,), dtype=float32)


<tf.Tensor: shape=(), dtype=float32, numpy=4.222307e-08>

The difference is very small (less than 10.0e-08)